# Setup

In [ ]:
from arcgis.gis import GIS

In [ ]:
agol = GIS("https://www.arcgis.com", "khibma_ncr")

# Portal Admin

### Note - depending on if AGOL or On-premise, it'll return a different property

###  What can we setup and customize?

In [ ]:
# We can also review and set some high level AGOL branding

desc = agol.admin.ux.description
orgname = agol.admin.ux.name

print(desc)
print(orgname)

In [ ]:
agol.admin.ux.description = "This is now Kevin's AGOL Org"
print(agol.admin.ux.description)

In [ ]:
agol.admin.ux.description = desc

In [ ]:
# Lets check the current logo..
agol.admin.ux.get_logo("D:/Technical/ArcGISPythonAPI_TechTrek/temp/")

In [ ]:
# Lets update it...
agol.admin.ux.set_logo("D:/Technical/ArcGISPythonAPI_TechTrek/temp/newLogo.jpg")

In [ ]:
# Better set it back
agol.admin.ux.set_logo("D:/Technical/ArcGISPythonAPI_TechTrek/temp/thumbnail.png")

Most of the "admin" things you'd do in AGOL are one time changes, like the description stuff above.
The `arcgis.gis.admin.AGOLAdminManager` object is better used to explain your AGOL Org 

reference: https://developers.arcgis.com/python/api-reference/arcgis.gis.admin.html#agoladminmanager

In [ ]:
# Check credits being burned every month...
agol.admin.credits.credit_usage()

In [ ]:
# Check a report to se credit burned by day...
agol.admin.usage_reports.credit()

In [ ]:
# What's the password policy?
agol.admin.password_policy.policy

In [ ]:
# Is tracking turned on?
agol.admin.location_tracking.status

In [ ]:
# Lets see if anyone is using it...
lastLocation = agol.admin.location_tracking.last_known_locations_layer

In [ ]:
from arcgis.features import FeatureLayer
ll_layer = FeatureLayer(lastLocation.url)

m = agol.map("Ottawa, Ontario")
m.add_layer(ll_layer)
m

And a few more limited things...

## A On-Prem Portal (and federated servers) offers more options

In [ ]:
# Don't need to do this
from arcgis.gis import GIS

In [ ]:
p = GIS("https://nogis21.esri.local/portal", "admin", "ags.admin") 
# Note - if using a self-signed or any untrusted certificate, can suppress SSL warnings and carry on
# verify_cert=False

In [ ]:
# How many servers are in this configuration?
servers = p.admin.servers.list()
print(servers)

In [ ]:
# What's been published to the only server?
s1 = servers[0]
print(s1)
s1_services = s1.services.list()
print(s1_services)

In [ ]:
# List data stores
for d in s1.datastores.list():
    print(d)

In [ ]:
# Sever logs
import pprint
logs = p.admin.logs.query(start_time = 1)
pprint.pprint(logs)

In [ ]:
# Dive into SSL
#  -properties
#  -list
#  -get
#  -import

p.admin.security.ssl

# Back to AGOL: Handling Users

In [ ]:
usrs = agol.users.search()
print("Found {} users in this org".format(len(usrs)))

In [ ]:
# Lets get a listing of all users and the last time they logged in
for u in usrs:
    print("{} last logged in: {}".format(u.username, u.lastLogin))

In [ ]:
# Dates are returned in epoch values; lets build a function to make nice dates
import datetime

def nice_date(d):
    return datetime.datetime.fromtimestamp(d/1000).strftime('%Y-%m-%d %H:%M')

In [ ]:
# Lets get a listing of all users and the last time they logged in
for u in usrs:
    print("{} last logged in: {}".format(u.username, nice_date(u.lastLogin)))

In [ ]:
# Find users who haven't logged in for 
inActivityThreshold = 30
# 86400 = milliseconds in a day
milliSecondThreshold = inActivityThreshold * 86400 * 1000
today = datetime.datetime.now().timestamp()


away_users = []
away_users_uObj = []
for u in usrs:
    if u.lastLogin < ((today * 1000) - milliSecondThreshold):
        print(u.username)
        away_users.append(u)
        away_users_uObj.append({
            'user': u.username,
            'email': u.email,
            'role': u.role,
            'login': nice_date(u.lastLogin)})


# Delete an old, inactive user

In [ ]:
for au in away_users:
    #au.delete() -- No, I better not
    pass

Why dont we email them?

In [ ]:
import smtplib
import sys
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

smtpServer = 'mySMTPSERVER.com'
sender = 'khibma@esri.ca'

def sendMail(to, subject, body):

    msg = MIMEMultipart()
    msg['From'] = sender
    msg['To'] = to 
    msg['Subject'] = subject

    theBody = body
    msg.attach( MIMEText(theBody) ) 
    
    try:
        smtpObj = smtplib.SMTP(smtpServer)        
        smtpObj.sendmail(sender, to, msg.as_string())
        smtpObj.quit

    except smtplib.SMTPException as e:
        sys.stderr.write("Error: unable to send email!\n")
        return False

    return True

In [ ]:
subject = "Are you still using AGOL "
body = "You haven't logged into AGOL in sometime. If you aren't using it, could you please email us and we'll reclaim your account."

# Send an email to each user in our object
for au in away_users_uObj:
    subject += au['user'] + '?'
    body += "\n You last logged in {}".format(au['login'])
    sendMail(au['email'], subject, body)

# Alternatively, we can handle inactive users completely in AGOL

In [ ]:
# Make a group to track them
inactv_grp = agol.groups.create(title="Inactive Users2", tags="techtrek, 2020")

In [ ]:
for au in away_users:
    inactv_grp.add_users(au)
        
print(inactv_grp.get_members())

In [ ]:
# Group has a notify option we can use to send them a note
inactv_grp.notify(users=['khibma_ncr'],
                 subject="You haven't logged into AGOL in sometime",
                 message="You've been added to a group of inactive users. Please login within 7 days or your account will be deleted.",
                 method="email")

# After 7 days we can above workflow and do a diff. Remove users from the inactive group or delete them

# Get all items in a group

In [ ]:
grps = agol.groups.search("title:Advanced Python for the Web", outside_org = False)

In [ ]:
print(grps)

In [ ]:
grp = grps[0]
for i in grp.content():
    print(i.title)

# Update apps in the group to HTTPS

In [ ]:
# A lot of the same concept's from Fabien's Update WebMap Content --
from arcgis.mapping import WebMap

def update_to_HTTPS(grp):
    
    # look through all group content
    for i in grp.content():        
        print("Processing: {}".format(i.title))
        
        # Only updating Web Maps
        if i.type == 'Web Map':
            wm = WebMap(i)
            
            # Look at all the layers in each webmap
            for idx, l in enumerate(wm.layers):
                
                # If the URL is NOT HTTP_S_ then update it
                if "HTTPS" not in l['url'].upper():
                    print("Found an HTTP layer that needs to be updated: \n   {}".format(l.url))
                    wm.layers[idx].url = wm.layers[idx].url.replace("http", "https")
                    print("Updated URL: {}".format(wm.layers[idx].url))
                    
                wm.update()
                    
update_to_HTTPS(grp)    
print("done")
            

# Show me more Administrative tricks!

### Ok... trusted servers, I need to add  them to my org to do local WAB development. 
### Where are those in the API?
### They aren't here, exposed through a property or method?

#### More on this later....  
reference: https://github.com/fabanc/esri-canada-tech-trek-2020/blob/master/AddAuthServers.py